In [2]:
#Token Intersection Ratio Score

In [1]:
def self_intersection_n(dataset, n, seed=0):
    """
    train data 용
    자기 자신을 n 분할하여 교집합 토큰을 구한다. 개수는 세지 않는다
    dataset: dataset 객체를 받는다
    n: 분할수
    seed: 데이터 random seed로 섞는다
    """
    # 데이터셋을 섞고 데이터 개수를 센다.
    dataset = dataset.shuffle(seed=seed)
    l = dataset.num_rows

    # 데이터개수를 n개로 슬라이싱하여 리스트에 담는다. 리스트에 담는 이유는 for문 돌릴려고
    # 빈 토큰 set을 n개만큼 생성하여 리스트에 담는다.
    data_split = [dataset[l*i//n:l*(i+1)//n] for i in range(n)]
    token_split = [set() for i in range(n)]

    # 슬라이싱된 집합을 돌면서 token들을 set에 넣는다
    for k in range(n):
        embeddings = tokenizer(data_split[k]['text'])['input_ids']
        for i in embeddings:
            for j in i:
                token_split[k].add(j)

    # 모든 set들의 교집합을 구히여 self 교집합을 구한다.
    self_intersection = token_split[0]
    for i in range(1,n):
        self_intersection = self_intersection & token_split[i]

    # 교집합을 반환한다.
    return self_intersection

In [2]:
def core_token_counter_n(dataset, intersection, n, seed=0):
    """
    valid data 용
    자기 자신을 n 분할하여 교집합 토큰을 구하고, 전체 교집합을 제한다. 개수도 저장한다.
    dataset: dataset 객체를 받는다
    intersection: 전체 교집합. self 교집합에서 제외하기 위해 받는다.
    n: 분할수 
    seed: 데이터 random seed로 섞는다
    """
    # 데이터셋을 섞고 데이터 개수를 센다.
    dataset = dataset.shuffle(seed=seed)
    l = dataset.num_rows
    
    # 토큰 개수를 저장할 딕셔너리를 생성한다.
    token_counter = {}
    # 코어 토큰만 저장할 딕셔너리를 생성한다.
    core = {}

    # 데이터개수를 n개로 슬라이싱하여 리스트에 담는다.
    # 빈 토큰 set을 n개만큼 생성하여 리스트에 담는다.
    data_split = [dataset[l*i//n:l*(i+1)//n] for i in range(n)]
    token_split = [set() for i in range(n)]

    # 슬라이싱된 데이터를 돌면서 set에도 넣으면서 빈도수도 딕셔너리에 저장한다.
    for k in range(n):
        embeddings = tokenizer(data_split[k]['text'])['input_ids']
        for i in embeddings:
            for j in i:
                token_split[k].add(j)
                if j in token_counter:
                    token_counter[j] += 1
                else:
                    token_counter[j] = 1

    # 분할 set끼리의 교집합을 구하여 self 교집합을 구한다.
    self_intersection = token_split[0]
    for i in range(1,n):
        self_intersection = self_intersection & token_split[i]

    # 딕셔너리를 돌면서 self 교집함에 존재하면 core 딕셔너리에 추가한다 (원래 딕셔너리에서 삭제하는 방법은 for문을 망가지게 한다)
    for i in token_counter:
        if i in self_intersection:
            core[i] = token_counter[i]

    # 코어 token과 빈도수를 가진 딕셔너리를 반환
    return core

In [31]:
from time import time
def compute_TRS(train1, train2, valid, n=3, seed=0):
    """
    점수를 계산한다.
    train1: 집단 1 데이터셋
    train2: 집단 2 데이터셋
    valid: 확인하고싶은 데이터셋
    n: 분할 수
    seed: 데이터 shuffle 랜덤 시드
    """
    start = time()
    # 집단 1,2 에 대해 self 교집합을 구한다.
    train1_intersection = self_intersection_n(train1, n, seed)
    train2_intersection = self_intersection_n(train2, n, seed)

    # 집단 1,2 self 교집합끼리의 교집합을 구해 전체 교집합을 구한다.
    intersection = train1_intersection & train2_intersection

    # 집단 1,2의 self 교집합에서 전체 교집합을 구해 각각의 core token을 구한다
    train1_core = train1_intersection - intersection
    train2_core = train2_intersection - intersection

    # 확인할 데이터셋의 core token을 구한다
    val_core = core_token_counter_n(valid, intersection, n, seed)

    # 점수 초기화
    score1 = 0
    score2 = 0

    # 확인할 데이터셋의 core token에서 token을 하나씩 꺼내와 집단 1,2 중 어느쪽에 존재하는지 확인한다. 없다면 패스 있다면 점수를 1점 올려준다.
    for i in val_core:
        if i in train1_core:
            score1 += 1
        elif i in train2_core:
            score2 += 1
    end = time()
    # 출력, 모종의 오류를 확인하기 위해 정답을 2번 그룹으로 두었다.
    print(score1, score2)
    print(f"n: {n} seed: {seed} | group1: {score1*100/(score1+score2)}% | group2: {score2*100/(score1+score2)} | {(end - start):.3f}sec")
    return score2*100/(score1+score2), end-start

In [11]:
'''
__________________________________________________

예시 코드
__________________________________________________
'''

'\n__________________________________________________\n\n예시 코드\n__________________________________________________\n'

In [5]:
# 토큰으로 찢기 위해 토크나이저를 불러옴. 어떤 것을 사용해도 상관없지만 BERT Tokenizer 사용. 다른거 사용해봐도 좋을듯
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [6]:
# 데이터셋 로드. 블로그에서 추출한 말뭉치 데이터셋 사용해보았음.
from datasets import load_dataset
dataset = load_dataset("blog_authorship_corpus")

In [7]:
# 16, 34살의 trian 데이터를 가져와 text만 뽑는다 
underage_data = dataset["train"].filter(lambda x: x["age"] == 16).select_columns(["text"])
old_data = dataset["train"].filter(lambda x: x["age"] == 34).select_columns(["text"])
# 16살 valid 데이터를 가져와 text만 뽑는다
underage_val = dataset["validation"].filter(lambda x: x["age"] == 16).select_columns(["text"])

In [33]:
# seed 0~30 까지로 shuffle 하면서 평균 점수 구해본다. 2중 포문으로 n도 늘려봐도 됨.
for n in range(30, 100, 5):
    meanS = 0
    meanT = 0
    for i in range(10):
        score, t = compute_TRS(old_data, underage_data, underage_val, n=n, seed=i)
        meanS += score
        meanT += t
    print(meanS/10, meanT/10)

0 5
n: 30 seed: 0 | group1: 0.0% | group2: 100.0 | 29.597sec
0 3
n: 30 seed: 1 | group1: 0.0% | group2: 100.0 | 30.215sec
0 9
n: 30 seed: 2 | group1: 0.0% | group2: 100.0 | 29.942sec
0 4
n: 30 seed: 3 | group1: 0.0% | group2: 100.0 | 29.580sec
0 4
n: 30 seed: 4 | group1: 0.0% | group2: 100.0 | 29.430sec
0 7
n: 30 seed: 5 | group1: 0.0% | group2: 100.0 | 29.773sec
0 6
n: 30 seed: 6 | group1: 0.0% | group2: 100.0 | 29.922sec
0 5
n: 30 seed: 7 | group1: 0.0% | group2: 100.0 | 29.873sec
0 5
n: 30 seed: 8 | group1: 0.0% | group2: 100.0 | 29.503sec
0 4
n: 30 seed: 9 | group1: 0.0% | group2: 100.0 | 30.003sec
100.0 29.783785700798035
0 7
n: 35 seed: 0 | group1: 0.0% | group2: 100.0 | 30.233sec
0 4
n: 35 seed: 1 | group1: 0.0% | group2: 100.0 | 29.778sec
0 4
n: 35 seed: 2 | group1: 0.0% | group2: 100.0 | 30.223sec
0 5
n: 35 seed: 3 | group1: 0.0% | group2: 100.0 | 29.898sec
0 5
n: 35 seed: 4 | group1: 0.0% | group2: 100.0 | 29.688sec
0 2
n: 35 seed: 5 | group1: 0.0% | group2: 100.0 | 29.542sec

KeyboardInterrupt: 

In [ ]:
'''
간단하게 n 마다 seed 10가지만 간략히 돌려본것. 100개씩은 돌려봐야 좀 신뢰도 있을거 같긴함.
1 85%
2 95%
3 96%
4 98.4%
5 98.2%
6 98.6%
7 99.6% (여기부터 거의 100% 뜨네)
8 99.5%
9 99.8%
10 99.8%
11 100%
12 100%
13 100%
14 99.2 (이상치 때문에) 근데 n을 늘릴수록 이상치가 나올 확률도 적어지는듯? 더 적게 뽑히는 대신 더 빡센 검증을 통해 뽑잖어
15 100 | 코어 20개 정도
16 100
17 99 | 코어 15개 정도 (1개만 반대로 갔는데 절대적인 수가 적어서 점수가 뚝떨어진다)
18 99
19 99
20 99
30 100 | 7개 (코어 뽑는게 빡세져서 슬슬 이상치도 안나오는듯 100까지 돌려보고싶다)
35 100 | 5개
40 100 | 3개
45 100 | 3개
50
55
확실히 3일때는 3:140 이정도였는데,14에서 0:20 정도로 떨어진다. 적게 뽑아도 확실한 코어 토큰을 뽑는게 나은건가
다음에는 비율말고 절대점수는 얼마나 떨어지는지 보자. 비율을 그대로여도 절대적인 점수는 떨어질거라고 생각한다.
근데 뭘 평가 하고싶은거지. bleu 같은 표준 llm 성능? 이건 그런쪽보다 GAN 평가방식 쪽 애들이랑 비교해야할듯 fid라던가.
